In [27]:
import pandas as pd
import joblib

# 1. Load the saved, pre-trained model
pipeline = joblib.load('final_crop_model.joblib')
print("Model loaded successfully!")

Model loaded successfully!


In [28]:
# 2. Define the target columns in the EXACT SAME ORDER as during training
# This is the crucial fix.
targets = ['total_duration_estimate'] + \
          [f'{stage}_{metric}' for stage in ['sw', 'sd_e', 'tilrr', 'stel', 'btng', 'hdng', 'flwr', 'grnm', 'grnd', 'rpng', 'hrvst'] for metric in ['tmin', 'tmax', 'rh', 'rain', 'wind', 'solar_rad']] + \
          [f'{stage}_stage_dur' for stage in ['sw', 'sd_e', 'tilrr', 'stel', 'btng', 'hdng', 'flwr', 'grnm', 'grnd', 'rpng', 'hrvst']]



In [29]:
# 2. Create a DataFrame with the new data for prediction
# The column names must exactly match the ones used during training
# Define the input data for the crop you want to predict
new_data = pd.DataFrame({
    'crop': ['Soybean'],
    'seed_type': ['JS 9560'],
    'soil': ['Medium Black (Vertisol)'],
    'district': ['Indore'],
    'season': ['Kharif']
})

In [30]:
# 4. Use the model to make predictions
predictions = pipeline.predict(new_data)

In [31]:
import json

from datetime import datetime
results_df = pd.DataFrame(predictions, columns=targets)

total_duration_pred = int(round(results_df['total_duration_estimate'].iloc[0]))
prediction_details_dict = {}
stages_codes = ['sw', 'sd_e', 'tilrr', 'stel', 'btng', 'hdng', 'flwr', 'grnm', 'grnd', 'rpng', 'hrvst']

for stage in stages_codes:
    prediction_details_dict[stage] = {
        "duration": int(round(results_df[f'{stage}_stage_dur'].iloc[0])),
        "tmin": results_df[f'{stage}_tmin'].iloc[0], "tmax": results_df[f'{stage}_tmax'].iloc[0],
        "rh": int(round(results_df[f'{stage}_rh'].iloc[0])), "rain": results_df[f'{stage}_rain'].iloc[0],
        "wind": results_df[f'{stage}_wind'].iloc[0], "solar_rad": int(round(results_df[f'{stage}_solar_rad'].iloc[0]))
    }

# --- Load Template and Populate with Prediction Data ---

template_path = 'wheat_time_line_data.json'
with open(template_path, 'r') as f:
    data = json.load(f)

stage_mapping = {
    "Sowing & Germination": "sw", "Seedling / Emergence": "sd_e", "Tillering": "tilrr",
    "Stem Elongation": "stel", "Booting": "btng", "Heading": "hdng", "Flowering": "flwr",
    "Grain Filling": "grnm", "Ripening": "rpng", "Harvest": "hrvst"
}

data['crop'] = new_data['crop'].iloc[0]
data['total_duration_days'] = total_duration_pred

for stage in data['stages']:
    stage_name = stage.get('name')
    if stage_name in stage_mapping:
        stage_code = stage_mapping[stage_name]
        if stage_code in prediction_details_dict:
            pred = prediction_details_dict[stage_code]
            stage['forecasted'] = {}
            stage['ideal'] = {
                "tmin_c": [pred["tmin"]], "tmax_c": [pred["tmax"]], "rh_pct": [pred["rh"]],
                "rain_mm": [pred["rain"]], "wind_kmph": [pred["wind"]], "solar_wm2": [pred["solar_rad"]]
            }
            stage['duration_days'] = pred["duration"]

# --- Create a Unique Filename and Save the JSON File ---

# Get current time for the timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
crop_name = new_data['crop'].iloc[0].lower()
district_name = new_data['district'].iloc[0].lower().replace(" ", "")

# Combine elements to create a unique, descriptive filename
output_filename = f"{crop_name}_{district_name}_{timestamp}.json"

with open(output_filename, 'w') as f:
    json.dump(data, f, indent=2)

print(f"\n✅ Success! Your prediction has been saved to the unique file: '{output_filename}'")


✅ Success! Your prediction has been saved to the unique file: 'soybean_indore_20251006_183817.json'
